## Input the Parameters of the Starlette-SLR GEODYN run:

In [1]:
import pandas as pd
import numpy as np

In [2]:
ll /data/runs_geodyn/iss/results/msis2/msis2_accelon/IIEOUT/


total 346680
-rw-rw-r--. 1 m_geodyn 354996479 Feb 22 22:40 iss190420_2dy.goco05s


In [31]:


########################
# INPUT PARAMETERS:
########################
sat_file = 'iss'
arc = '190420_2dy'
grav_id ='goco05s' 
local_path = '/data/analysis/ISS_GPS_analysis/'
SAT_ID_iss = 9806701
SAT_ID_slr =7501001
accel_card = 'accelon'


#######################################
# PATH TO DENSITY MODEL RUN of Choice:
#######################################
msis2_model = 'msis2'



### Call the prep-function that loads the data

This may take a few minutes to load.

#### MSIS2

In [19]:
path_to_iss = '/data/runs_geodyn/'+sat_file+'/results/'+ msis2_model+'/'+ msis2_model+ '_' +accel_card+'/'+'IIEOUT/iss190420_2dy.goco05s'
path_to_slr = '/data/runs_geodyn/'+'st'+'/results/'+ msis2_model+'/'+ msis2_model+ '_' +accel_card+'/'+ 'IIEOUT/st030914_2wk.goco05s'


In [20]:
path_to_slr

'/data/runs_geodyn/st/results/msis2/msis2_accelon/IIEOUT/st030914_2wk.goco05s'

In [110]:
# WONKY INPUTS:
AccelStatus = True


sat = sat_file
file_name =  sat + arc + '.'+ grav_id
# print('The base file name for this arc is:',file_name,'\n' )
import os

iieout_file  = path_to_iss
# iieout_file  = path_to_slr
SAT_ID = SAT_ID_iss

# iieout_file  = '/data/analysis/IIEOUT_'+file_name


# from b_ReadISS import Save_AdjustedParameters_ISS
# SatMain_AdjustedParams = Save_AdjustedParameters_ISS(SAT_ID, iieout_file, AccelStatus)


In [150]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta
import os.path
import linecache

Sat_main = SAT_ID
iieout_file
AccelStatus =True
DATA_TYPE = 'GPS'

In [153]:


def Save_AdjustedParameters_ISS(Sat_main, iieout_file, AccelStatus = True, DATA_TYPE):
    '''
    The below code grabs the estimated adjusted parameters for the MAIN satellite for
    the each iteration.
    The data is stored in a dictionary and includes the associated statistics:
          APRIORI  VALUE
          PREVIOUS VALUE              
          CURRENT  VALUE              
          TOTAL DELTA
          CURRENT DELTA
          APRIORI SIGMA
          CURRENT SIGMA
    The dictionary is multi-dimensional, with the first dimension being iteration 
    number and second dimension being satellite ID, the third dim contains the above data.

            ADJUSTEDPARAMS[iteration][SatID][param][stat_value]

    #################################################################################

    The way that this function works is it basically loops through the MASSIVE IIEOUT
    file and finds the line numbers for certain keywords (0XPOS, 0YPOS, ... etc.).
    We then grab the data next to each of the keywords (the format is fixed).

    If we are looking at GPS tracked data, we must identify the satellite ID that will be next to each set ofkeywords


    EXAMPLE: the below block of code will be indexed such that the line number for 
             each 0XPOS, 0YPOS, ... etc. is known.



                1ARC  1 PARAMETER ADJUSTMENT SUMMARY FOR INNER ITERATION  6 OF GLOBAL ITERATION 1    
                0 PARAMETER NAME       APRIORI  VALUE
                                       PREVIOUS VALUE         TOTAL   DELTA     APRIORI SIGMA
                                       CURRENT  VALUE         CURRENT DELTA     CURRENT SIGMA
                0XPOS     5041144  -21439971.75446300
                                   -21439971.75446300       0.00000000000      0.10000000E-12
                                   -21439971.75446300       0.00000000000      0.10000000E-12

                0YPOS     5041144  -13434973.98958000
                                   -13434973.98958000       0.00000000000      0.10000000E-12
                                   -13434973.98958000       0.00000000000      0.10000000E-12

                0ZPOS     5041144   8698413.759462100
                                    8698413.759462100       0.00000000000      0.10000000E-12
                                    8698413.759462100       0.00000000000      0.10000000E-12

                0XVEL     5041144   2056.571120089500
                                    2056.571120089500       0.00000000000      0.10000000E-12
                                    2056.571120089500       0.00000000000      0.10000000E-12

                0YVEL     5041144  -1459.475867380500
                                   -1459.475867380500       0.00000000000      0.10000000E-12
                                   -1459.475867380500       0.00000000000      0.10000000E-12

                0ZVEL     5041144   2903.476794151500
                                    2903.476794151500       0.00000000000      0.10000000E-12
                                    2903.476794151500       0.00000000000      0.10000000E-12

    '''


    # find the line numbers of all the adjustment parameters in the too big iieout file
    text_param_lists = ['0XPOS',
                        '0YPOS',
                        '0ZPOS',
                        '0XVEL',
                        '0YVEL',
                        '0ZVEL',]


    # Identify the lines where we have changed parameters and
    # Make a list of the satellites that have changed parameters in the file
    # Only add sat id's to list if they are not accounted for.
    sat_list = []
    lines_params = [] 
    with open(iieout_file, 'r') as f:
        for line_no, line_text in enumerate(f):
            if '0XPOS' in line_text :
                lines_params.append(line_no)
                if int(linecache.getline(iieout_file,line_no+1)[10:18]) not in sat_list:
                    sat_list.append(int(linecache.getline(iieout_file,line_no+1)[10:18]))
                else:
                    pass

    ####                
    # identify how many iteration numbers there are.           
    with open(iieout_file, 'r') as f:
        for line_no, line in enumerate(f):
            if 'CONVERGENCE' in line:
                line_text = line
    num_iters = float(line_text[39:42])-1

    # Build the dictionary to be index based on iteration number   
    # and initialize each iteration number to also be a dictionary
    text_GA_list = ["0GA 9P 11",
                    "0GA 9P 12",
                    "0GA 9P 21",
                    "0GA 9P 22",
                        ]    
    SatMain_AdjustedParams = {}
    for i_iter,iterval in enumerate(np.arange(1, int(num_iters)+1)):
        SatMain_AdjustedParams[iterval] = {}
        for isat, satval in enumerate(sat_list):
            for iga, ga_val in enumerate(text_GA_list):
                SatMain_AdjustedParams[iterval][satval] = {}
                SatMain_AdjustedParams[iterval][satval]['0CD'] = {}
                SatMain_AdjustedParams[iterval][satval][ga_val] = {}
    # 
    # make a list of the dates for the time dependent drag option
    # First, determine how many time dependent outputs there are:
    #
    line_no_1 = [] 
    line_no_2 = [] 
    with open(iieout_file, 'r') as f:
        for line_no, line_text in enumerate(f):
            if ' ARC #    1   PARAMETERS' in line_text :
                line_no_1.append(line_no)
            elif '        GLOBAL PARAMETER SUMMARY' in line_text:
                line_no_2.append(line_no)
    #            
    parameter_summary_section_range = np.arange(line_no_1[0], line_no_2[0]+1)
    #
    # Count how many Time dep Cd's were processed
    timedep_Cd_count = []
    for i,val in enumerate(parameter_summary_section_range):
            line = linecache.getline(iieout_file,val)            
            if 'CD' in line:
                check_sat = int(line[24:32])
                if check_sat == Sat_main:
                    if 'T' in line:
                        timedep_Cd_count.append(line[18:24].strip()) # save the list of T##s and strip of whitespaces
    # Loop through the IIE cards to find the
    # date inputs for the time dependent Cd option 
    # First isolate the input card section:
    line_no_1 = [] 
    line_no_2 = [] 
    with open(iieout_file, 'r') as f:
        for line_no, line_text in enumerate(f):
            if 'GEODYN IIE VERSION' in line_text :
                line_no_1.append(line_no)
            if 'OBSERVATION RESIDUALS FOR ARC' in line_text:
                line_no_2.append(line_no)
                break
    # Make a list of the dates as determined by the DRAG input cards:            
    card_inputs_range = np.arange(line_no_1[0], line_no_2[0]-100)
    timedep_Cd_dates = []
    for i,val in enumerate(card_inputs_range):
            line = linecache.getline(iieout_file,val)            
            if 'DRAG' in line:
                check_sat = int(line[18:26])
                if check_sat == Sat_main:

    #                 print(line[45:57])
                    timedep_Cd_dates.append(line[45:56].strip())
    date_timedep_cds = pd.to_datetime(timedep_Cd_dates[1:], format='%y%m%d%H%M%S')  #YYMMDDHHMMSS


    # Search through the full file for the key words in the above list (text_param_lists)
    # save the line numbers where those keywords occur
    lines = []
    for text_param_adjusts in text_param_lists: 
        with open(iieout_file, 'r') as f:
            for line_no, line_text in enumerate(f):
                if text_param_adjusts in line_text:
                    lines.append(line_no) 

        # Loop thru the lines saved above and grab the data occording to its name 
        # and location in the file
    #     Sat_main_lines = [] 
        i=0
        for il,val_lines in enumerate(lines):
            check_sat = int(linecache.getline(iieout_file,val_lines+1)[10:18])
            try:
                check_iter = int((linecache.getline(iieout_file,lines_params[i]-3))[57:60])
            except:
                check_iter = int((linecache.getline(iieout_file,lines_params[i]-27))[57:60])
    #         print('Iter: ', check_iter)

            data_1stline = linecache.getline(iieout_file,val_lines+1) #
            data_2ndtline = linecache.getline(iieout_file,val_lines+2) #
            data_3rdline = linecache.getline(iieout_file,val_lines+3) #

            apriorival = float(data_1stline[19:38])
            prevval = float(data_2ndtline[19:38])
            currentval  = float(data_3rdline[19:38])
            totalDelta = float(data_2ndtline[42:62])
            currentDelta =  float(data_3rdline[42:62])
            AprioriSigma = float(data_2ndtline[63:78])
            CurrentSigma =  float(data_3rdline[63:78])

            SatMain_AdjustedParams[check_iter][check_sat][text_param_adjusts] = {'APRIORI_VALUE': apriorival,
                                                 'PREVIOUS_VALUE': prevval,
                                                 'CURRENT_VALUE': currentval,
                                                  'TOTAL_DELTA':totalDelta,
                                                 'CURRENT_DELTA': currentDelta,
                                                 'APRIORI_SIGMA': AprioriSigma,
                                                 'CURRENT_SIGMA': CurrentSigma }
            i+=1
            i = np.mod(i,np.shape(lines_params)[0])  # this makes it so that you can properly index the iterations

    # Create a list of the number TimeDep drag coeffiecient headers
    text_cd_list = ["0CD   T%02d" %i for i in np.arange(1,np.size(timedep_Cd_count)+1 )]

    lines = []
    for itt,text_param_adjusts in enumerate(text_cd_list): 
    #     print(itt, text_param_adjusts)
        with open(iieout_file, 'r') as f:
            for line_no, line_text in enumerate(f):
                if text_param_adjusts in line_text:
                    lines.append(line_no) 

        # Loop thru the lines saved above and grab the data occording to its name 
        # and location in the file
        for il,val_lines in enumerate(lines):
            check_sat = int(linecache.getline(iieout_file,val_lines+1)[10:18])
    #         check_iter = int((linecache.getline(iieout_file,val_lines+1-28))[57:60])

            # Read the lines backwars until you hit a header
            find_last_header_range = np.arange(val_lines, val_lines-1000, -1)
            for iiline, iivaline in enumerate(find_last_header_range):
                line_find_header = linecache.getline(iieout_file,iivaline)
                if 'PARAMETER ADJUSTMENT SUMMARY' in line_find_header:
                    check_iter =int(line_find_header[57:60])
                    break

            data_1stline = linecache.getline(iieout_file,val_lines+1) #
            data_2ndtline = linecache.getline(iieout_file,val_lines+2) #
            data_3rdline = linecache.getline(iieout_file,val_lines+3) #

            apriorival = float(data_1stline[19:38])
            prevval = float(data_2ndtline[19:38])
            currentval  = float(data_3rdline[19:38])
            totalDelta = float(data_2ndtline[42:62])
            currentDelta =  float(data_3rdline[42:62])
            AprioriSigma = float(data_2ndtline[63:78])
            CurrentSigma =  float(data_3rdline[63:78])

    #         print('Sat:  ', check_sat)
    #         print('Iter: ', check_iter)
    #         print('Val:  ', currentval)
    #         print('Time: ', date_timedep_cds[itt])

            SatMain_AdjustedParams[check_iter][check_sat]['0CD'][date_timedep_cds[itt]] = {'APRIORI_VALUE': apriorival,
                                                                    'PREVIOUS_VALUE': prevval,
                                                                    'CURRENT_VALUE': currentval,
                                                                    'TOTAL_DELTA':totalDelta,
                                                                    'CURRENT_DELTA': currentDelta,
                                                                    'APRIORI_SIGMA': AprioriSigma,
                                                                    'CURRENT_SIGMA': CurrentSigma }



    if AccelStatus == True:
        if DATA_TYPE == 'GPS':
            print('There are some weird things here for Gen.Acc. Not done yet.')
    #         break
        else:

            accel_9_dates = []
            for i,val in enumerate(card_inputs_range):
                    line = linecache.getline(iieout_file,val)            
                    if 'ACCEL9    99' in line:
                        check_sat = int(line[18:26])
                        if check_sat == Sat_main:

                            accel_9_dates.append(line[45:56].strip())
            #------------------------------------------------------------------            
            # this block of code is used to remove the whitespace from the date 
            fix_date_string = []
            for i, val in enumerate(accel_9_dates):
                if ' ' in val:
                    rebuild_string = []
                    for ii,valval in enumerate(val):
                        if valval == ' ':
                            rebuild_string.append(valval.replace(" ", "0"))
                        else:
                            rebuild_string.append(valval)
                    rebuild_string = "".join(rebuild_string)
                    fix_date_string.append(rebuild_string)
                else:
                    fix_date_string.append(val)
        #------------------------------------------------------------------            
            accel_9_Dates = pd.to_datetime(fix_date_string, format='%y%m%d%H%M%S')  #YYMMDDHHMMSS

                # AND THIS SPECIFIC LIST OF Gen. ACCELS
                # 1st Value (1 or 2) Indicates direction of 9 parameter general acceleration
                    # 1 - Along Track ((R x V) x R)
                    # 2 - Cross Track (R x V)
                    # 3 - Radial (R)

                #  2nd Value (1 or 2) Indicates type of 9 parameter general acceleration parameter
                    # 1 - Cosine coefficient (A)
                    # 2 - Sine coefficient (B)
                    # 3 - Constant (C)
            text_GA_list = ["0GA 9P 11",
                            "0GA 9P 12",
                            "0GA 9P 21",
                            "0GA 9P 22",
                            ]    


            lines = []
            for itt,text_GA_adjusts in enumerate(text_GA_list): 

                with open(iieout_file, 'r') as f:
                    for line_no, line_text in enumerate(f):
                        if text_GA_adjusts in line_text:
                            lines.append(line_no) 

                # Loop thru the lines saved above and grab the data occording to its name 
                # and location in the file
                for il,val_lines in enumerate(lines[::2]):
                    check_sat = int(linecache.getline(iieout_file,val_lines+1)[10:18])

                    # Read the lines backwars until you hit a header
                    find_last_header_range = np.arange(val_lines, val_lines-1000, -1)
                    for iiline, iivaline in enumerate(find_last_header_range):
                        line_find_header = linecache.getline(iieout_file,iivaline)
                        if 'PARAMETER ADJUSTMENT SUMMARY' in line_find_header:
                            check_iter =int(line_find_header[57:60])
                            break

                    data_1stline = linecache.getline(iieout_file,val_lines+1) #
                    data_2ndtline = linecache.getline(iieout_file,val_lines+2) #
                    data_3rdline = linecache.getline(iieout_file,val_lines+3) #

                    apriorival = float(data_1stline[18:41])
                    prevval = float(data_2ndtline[18:41])
                    currentval  = float(data_3rdline[18:41])
                    totalDelta = float(data_2ndtline[42:62])
                    currentDelta =  float(data_3rdline[42:62])
                    AprioriSigma = float(data_2ndtline[63:78])
                    CurrentSigma =  float(data_3rdline[63:78])

            #         print('Sat:  ', check_sat)
            #         print('Iter: ', check_iter)
            #         print('Val:  ', currentval)
            #         print('Time: ', date_timedep_cds[itt])

                    SatMain_AdjustedParams[check_iter][check_sat][text_GA_adjusts][accel_9_Dates[0]] = {'APRIORI_VALUE': apriorival,
                                                                            'PREVIOUS_VALUE': prevval,
                                                                            'CURRENT_VALUE': currentval,
                                                                            'TOTAL_DELTA':totalDelta,
                                                                            'CURRENT_DELTA': currentDelta,
                                                                            'APRIORI_SIGMA': AprioriSigma,
                                                                            'CURRENT_SIGMA': CurrentSigma }
                for il,val_lines in enumerate(lines[1::2]):
                    check_sat = int(linecache.getline(iieout_file,val_lines+1)[10:18])

                    # Read the lines backwars until you hit a header
                    find_last_header_range = np.arange(val_lines, val_lines-1000, -1)
                    for iiline, iivaline in enumerate(find_last_header_range):
                        line_find_header = linecache.getline(iieout_file,iivaline)
                        if 'PARAMETER ADJUSTMENT SUMMARY' in line_find_header:
                            check_iter =int(line_find_header[57:60])
                            break

                    data_1stline = linecache.getline(iieout_file,val_lines+1) #
                    data_2ndtline = linecache.getline(iieout_file,val_lines+2) #
                    data_3rdline = linecache.getline(iieout_file,val_lines+3) #

                    apriorival = float(data_1stline[18:41])
                    prevval = float(data_2ndtline[18:41])
                    currentval  = float(data_3rdline[18:41])
                    totalDelta = float(data_2ndtline[42:62])
                    currentDelta =  float(data_3rdline[42:62])
                    AprioriSigma = float(data_2ndtline[63:78])
                    CurrentSigma =  float(data_3rdline[63:78])

            #         print('Sat:  ', check_sat)
            #         print('Iter: ', check_iter)
            #         print('Val:  ', currentval)
            #         print('Time: ', date_timedep_cds[itt])

                    SatMain_AdjustedParams[check_iter][check_sat][text_GA_adjusts][accel_9_Dates[1]] = {'APRIORI_VALUE': apriorival,
                                                                            'PREVIOUS_VALUE': prevval,
                                                                            'CURRENT_VALUE': currentval,
                                                                            'TOTAL_DELTA':totalDelta,
                                                                            'CURRENT_DELTA': currentDelta,
                                                                            'APRIORI_SIGMA': AprioriSigma,
                                                                            'CURRENT_SIGMA': CurrentSigma }


    return(SatMain_AdjustedParams)

There are some weird things here for Gen.Acc. Not done yet.


DatetimeIndex(['2019-04-20 21:24:01', '2019-04-20 21:48:02',
               '2019-04-20 22:12:03', '2019-04-20 22:36:04',
               '2019-04-20 23:00:05', '2019-04-20 23:25:00',
               '2019-04-20 23:49:01', '2019-04-21 00:13:02',
               '2019-04-21 00:37:03', '2019-04-21 01:01:04',
               ...
               '2019-04-22 23:45:00', '2019-04-23 00:09:01',
               '2019-04-23 00:33:02', '2019-04-23 00:57:03',
               '2019-04-23 01:21:04', '2019-04-23 01:45:05',
               '2019-04-23 02:10:00', '2019-04-23 02:34:01',
               '2019-04-23 02:58:02', '2019-04-23 05:00:00'],
              dtype='datetime64[ns]', length=135, freq=None)

In [121]:
accel_9_Dates

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

190421001320
190421003730
190421010140
190421012550
190421015000
190421021410
190421023820
190421030230
190421032640
190421035050
190421041500
190421043910
190421050320
190421052730
190421055140
190421061550
190421064000
190421070410
190421072820
190421075230
190421081640
190421084050
190421090500
190421092910
190421095320
190422002320
190422004730
190422011140
190422013550
190422020000
190422022410
190422024820
190422031230
190422033640
190422040050
190422042500
190422044910
190422051320
190422053730
190422060140
190422062550
190422065000
190422071410
190422073820
190422080230
190422082640
190422085050
190422091500
190422093910
190423000910
190423003320
190423005730
190423012140
190423014550
190423021000
190423023410
190423025820
190423050000
0 190420212410
1 190420214820
2 190420221230
3 190420223640
4 190420230050
5 190420232500
6 190420234910
7 190421  1320
8 190421  3730
9 190421 10140
10 190421 12550
11 190421 15000
12 190421 21410
13 190421 23820
14 190421 30230
15 190421 32640


In [147]:
for i, val in enumerate(fix_date_string):
    print(i,val)


0 190420212410
1 190420214820
2 190420221230
3 190420223640
4 190420230050
5 190420232500
6 190420234910
7 190421001320
8 190421003730
9 190421010140
10 190421012550
11 190421015000
12 190421021410
13 190421023820
14 190421030230
15 190421032640
16 190421035050
17 190421041500
18 190421043910
19 190421050320
20 190421052730
21 190421055140
22 190421061550
23 190421064000
24 190421070410
25 190421072820
26 190421075230
27 190421081640
28 190421084050
29 190421090500
30 190421092910
31 190421095320
32 190421101730
33 190421104140
34 190421110550
35 190421113000
36 190421115410
37 190421121820
38 190421124230
39 190421130640
40 190421133050
41 190421135500
42 190421141910
43 190421144320
44 190421150730
45 190421153140
46 190421155550
47 190421162000
48 190421164410
49 190421170820
50 190421173230
51 190421175640
52 190421182050
53 190421184500
54 190421190910
55 190421193320
56 190421195730
57 190421202140
58 190421204550
59 190421211000
60 190421213410
61 190421215820
62 190421222230
63

{Timestamp('2003-09-14 08:00:00'): {'APRIORI_VALUE': 2.2,
  'PREVIOUS_VALUE': 2.768337262080763,
  'CURRENT_VALUE': 2.76833775274191,
  'TOTAL_DELTA': 0.568337752742,
  'CURRENT_DELTA': 4.90661146656e-07,
  'APRIORI_SIGMA': 10.0,
  'CURRENT_SIGMA': 0.37012613},
 Timestamp('2003-09-14 16:00:00'): {'APRIORI_VALUE': 2.2,
  'PREVIOUS_VALUE': 2.670970704283059,
  'CURRENT_VALUE': 2.670970818225002,
  'TOTAL_DELTA': 0.470970818225,
  'CURRENT_DELTA': 1.13941942797e-07,
  'APRIORI_SIGMA': 10.0,
  'CURRENT_SIGMA': 0.22297213},
 Timestamp('2003-09-15 00:00:00'): {'APRIORI_VALUE': 2.2,
  'PREVIOUS_VALUE': 2.885332767587743,
  'CURRENT_VALUE': 2.885335166486637,
  'TOTAL_DELTA': 0.685335166487,
  'CURRENT_DELTA': 2.39889889402e-06,
  'APRIORI_SIGMA': 10.0,
  'CURRENT_SIGMA': 0.17757554},
 Timestamp('2003-09-15 08:00:00'): {'APRIORI_VALUE': 2.2,
  'PREVIOUS_VALUE': 3.337599213589603,
  'CURRENT_VALUE': 3.337604207789252,
  'TOTAL_DELTA': 1.13760420779,
  'CURRENT_DELTA': 4.9941996491e-06,
  'APRIO

In [10]:
# # Create a list of the number TimeDep drag coeffiecient headers
# # FOR NOW THIS IS HARDCODED IN AS 43 CD VALUES....
# text_cd_list = ["0CD   T%02d" %i for i in np.arange(1,43)]

# for itt,text_param_adjusts in enumerate(text_cd_list): 
#     lines = []                

#     with open(iieout_file, 'r') as f:
#         for line_no, line_text in enumerate(f):
#             if text_param_adjusts in line_text:
#                 lines.append(line_no)

#     # Loop thru the lines saved above and grab the data occording to its name 
#     # and location in the file
#     Sat_main_lines = [] 
#     i=0
#     for il,val_lines in enumerate(lines):
#         check_sat = int(linecache.getline(iieout_file,val_lines+1)[10:18])
#         check_iter = int((linecache.getline(iieout_file,lines_params[i]-3))[57:60])
#         if check_sat == Sat_main:
#             # print(check_sat)
#             # print(check_iter)
#             data_1stline = linecache.getline(iieout_file,val_lines+1) #
#             data_2ndtline = linecache.getline(iieout_file,val_lines+2) #
#             data_3rdline = linecache.getline(iieout_file,val_lines+3) #

#             apriorival = float(data_1stline[19:38])
#             prevval = float(data_2ndtline[19:38])
#             currentval  = float(data_3rdline[19:38])
#             totalDelta = float(data_2ndtline[42:62])
#             currentDelta =  float(data_3rdline[42:62])
#             AprioriSigma = float(data_2ndtline[63:78])
#             CurrentSigma =  float(data_3rdline[63:78])

#             SatMain_AdjustedParams[check_iter]['0CD'][date_timedep_cds[itt]] = {'APRIORI_VALUE': apriorival,
#                                                  'PREVIOUS_VALUE': prevval,
#                                                  'CURRENT_VALUE': currentval,
#                                                   'TOTAL_DELTA':totalDelta,
#                                                  'CURRENT_DELTA': currentDelta,
#                                                  'APRIORI_SIGMA': AprioriSigma,
#                                                  'CURRENT_SIGMA': CurrentSigma }
#             i+=1
#             i = np.mod(i,np.shape(lines_params)[0])

        else:
            print('Oops! Grabbing data for the wrong satellite: ', str(check_sat))
            continue

if AccelStatus == True:

    # AND THIS SPECIFIC LIST OF Gen. ACCELS
    # 1st Value (1 or 2) Indicates direction of 9 parameter general acceleration
        # 1 - Along Track ((R x V) x R)
        # 2 - Cross Track (R x V)
        # 3 - Radial (R)

    #  2nd Value (1 or 2) Indicates type of 9 parameter general acceleration parameter
        # 1 - Cosine coefficient (A)
        # 2 - Sine coefficient (B)
        # 3 - Constant (C)
    text_GA_list = ["0GA 9P 11",
                    "0GA 9P 12",
                    "0GA 9P 21",
                    "0GA 9P 22",
                   ]    

    for text_param_adjusts in text_GA_list: 
        lines = []  
        with open(iieout_file, 'r') as f:
            for line_no, line_text in enumerate(f):
                if text_param_adjusts in line_text:
                    lines.append(line_no)


        # Loop thru the lines saved above and grab the data occording to its name 
        # and location in the file
        Sat_main_lines = [] 
        i=0
        for il,val_lines in enumerate(lines[::2]):
            check_sat = int(linecache.getline(iieout_file,val_lines+1)[10:18])
            check_iter = int((linecache.getline(iieout_file,lines_params[i]-3))[57:60])
            if check_sat == Sat_main:
                # print(check_sat)
                # print(check_iter)
                data_1stline = linecache.getline(iieout_file,val_lines+1) #
                data_2ndtline = linecache.getline(iieout_file,val_lines+2) #
                data_3rdline = linecache.getline(iieout_file,val_lines+3) #

                apriorival = float(data_1stline[19:42])
                prevval = float(data_2ndtline[19:42])
                currentval  = float(data_3rdline[19:42])
                totalDelta = float(data_2ndtline[42:63])
                currentDelta =  float(data_3rdline[42:63])
                AprioriSigma = float(data_2ndtline[63:79])
                CurrentSigma =  float(data_3rdline[63:79])


                SatMain_AdjustedParams[check_iter]['0GA'][text_param_adjusts] = {'APRIORI_VALUE': apriorival,
                                                     'PREVIOUS_VALUE': prevval,
                                                     'CURRENT_VALUE': currentval,
                                                      'TOTAL_DELTA':totalDelta,
                                                     'CURRENT_DELTA': currentDelta,
                                                     'APRIORI_SIGMA': AprioriSigma,
                                                     'CURRENT_SIGMA': CurrentSigma }
                i+=1
                i = np.mod(i,np.shape(lines_params)[0])

            else:
                print('Oops! Grabbing data for the wrong satellite: ', str(check_sat))
                continue
        for il,val_lines in enumerate(lines[1::2]):
            check_sat = int(linecache.getline(iieout_file,val_lines+1)[10:18])
            check_iter = int((linecache.getline(iieout_file,lines_params[i]-3))[57:60])
            if check_sat == Sat_main:
                print(check_sat)
                print(check_iter)
                data_1stline = linecache.getline(iieout_file,val_lines+1) #
                data_2ndtline = linecache.getline(iieout_file,val_lines+2) #
                data_3rdline = linecache.getline(iieout_file,val_lines+3) #

                apriorival = float(data_1stline[19:42])
                prevval = float(data_2ndtline[19:42])
                currentval  = float(data_3rdline[19:42])
                totalDelta = float(data_2ndtline[42:63])
                currentDelta =  float(data_3rdline[42:63])
                AprioriSigma = float(data_2ndtline[63:79])
                CurrentSigma =  float(data_3rdline[63:79])


                SatMain_AdjustedParams[check_iter]['0GA_t2'][text_param_adjusts] = {'APRIORI_VALUE': apriorival,
                                                     'PREVIOUS_VALUE': prevval,
                                                     'CURRENT_VALUE': currentval,
                                                      'TOTAL_DELTA':totalDelta,
                                                     'CURRENT_DELTA': currentDelta,
                                                     'APRIORI_SIGMA': AprioriSigma,
                                                     'CURRENT_SIGMA': CurrentSigma }
                i+=1
                i = np.mod(i,np.shape(lines_params)[0])

            else:
                print('Grabbing data for the GPS satellite: ', str(check_sat))
                continue
else:
    pass
#     print('Done')
# print('Done saving the following parameters: \n ', np.squeeze(text_cd_list))
# return(SatMain_AdjustedParams)

IndentationError: unexpected indent (<ipython-input-10-123983106379>, line 45)